# ***Importing libraries***

In [27]:
import json
import string
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re 
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import tensorflow as tf 
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.convolutional import Conv1D,MaxPooling1D 
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Input,GlobalAveragePooling1D,Flatten, Activation, Dropout
from keras.layers.wrappers import Bidirectional
from keras.models import Model
import json
import pickle
import numpy as np
from tensorflow.keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to C:\Users\Farah Ben
[nltk_data]     Jannette\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Farah Ben
[nltk_data]     Jannette\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
lemmatizer = WordNetLemmatizer()

# ***Loading Data***

In [ ]:
data_file = open('data.json').read()
intents = json.loads(data_file)

# ***NLTK /Data preprocessing***

In [55]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)


44 documents
7 classes [' group_info', 'about', 'goodbye', 'greeting ', 'name', 'options', 'thanks']
51 unique lemmatized words ["'s", ')', ',', ':', 'about', 'anyone', 'are', 'awesome', 'bye', 'call', 'can', 'chaabane', 'chaaben', 'day', 'do', 'ekse', 'for', 'gm', 'good', 'goodbye', 'group', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'is', 'know', 'later', 'me', 'more', 'name', 'ok', 'purpose', 'see', 'should', 'technology', 'tell', 'thank', 'thanks', 'that', 'there', 'what', 'whats', 'who', 'you', 'your']
Training data created


<ipython-input-55-45f49a0929e9>:63: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


# ***Data preparation***

In [ ]:
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

# ***combined cnn_Bilstm model***

In [56]:
#hyperparameters 
vocaburaly=len(documents)
input_shape=(len(train_x[0]))
output_shape=len(train_y[0])
unit=50
batch_size=5

In [57]:
model = Sequential()
model.add(Embedding(vocaburaly,10,input_length=input_shape))
model.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='causal'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.7))

model.add(Bidirectional(LSTM(units=unit)))
model.add(Dropout(0.7))
model.add(Dense(output_shape), activation ='softmax'))
model.summary()


Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 51, 10)            440       
                                                                 
 conv1d_14 (Conv1D)          (None, 51, 64)            3264      
                                                                 
 max_pooling1d_11 (MaxPoolin  (None, 25, 64)           0         
 g1D)                                                            
                                                                 
 dropout_18 (Dropout)        (None, 25, 64)            0         
                                                                 
 bidirectional_2 (Bidirectio  (None, 100)              46000     
 nal)                                                            
                                                                 
 dropout_19 (Dropout)        (None, 100)             

# compile and fit model 

In [58]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=batch_size, verbose=1)


Epoch 1/300
9/9 [==============================] - 5s 13ms/step - loss: 1.9345 - accuracy: 0.2273
Epoch 2/300
9/9 [==============================] - 0s 14ms/step - loss: 1.9123 - accuracy: 0.1591
Epoch 3/300
9/9 [==============================] - 0s 13ms/step - loss: 1.8541 - accuracy: 0.1591
Epoch 4/300
9/9 [==============================] - 0s 13ms/step - loss: 1.8682 - accuracy: 0.2045
Epoch 5/300
9/9 [==============================] - 0s 12ms/step - loss: 1.8863 - accuracy: 0.2273
Epoch 6/300
9/9 [==============================] - 0s 14ms/step - loss: 1.8445 - accuracy: 0.2727
Epoch 7/300
9/9 [==============================] - 0s 11ms/step - loss: 1.8803 - accuracy: 0.2045
Epoch 8/300
9/9 [==============================] - 0s 12ms/step - loss: 1.7760 - accuracy: 0.3182
Epoch 9/300
9/9 [==============================] - 0s 12ms/step - loss: 1.8860 - accuracy: 0.1818
Epoch 10/300
9/9 [==============================] - 0s 13ms/step - loss: 1.7798 - accuracy: 0.3409
Epoch 11/300
9/9 [=

In [59]:
model.save('model.h5', hist)
print("model created")

model created
